In [ ]:
"""
SAP Report Automation - Optimized for Performance
Improvements:
1. All imports at top (avoid repeated import overhead in loops)
2. Reduced sleep times where safe (total savings ~30+ seconds)
3. Pre-computed date values (avoid repeated datetime calculations)
4. Constants for repeated strings
5. Helper functions for better organization and reusability
6. Optimized Excel closing with retry logic
"""
import os
import subprocess
import sys
import time
import traceback
from datetime import datetime
from calendar import monthrange
from typing import Optional

import win32com.client

# ============== CONSTANTS (避免重复字符串创建) ==============
SAP_TRANSACTION = "S_ALR_87011990"
SAP_SERVER = "RPR - ECC Production"
BASE_SAVE_PATH = r"\\apac.wdpr.disney.com\Corp\CNKC\Finance\Controllership\FINANCIAL REPORTING\FY_ 2026\Month-End"

# SAP GUI 可能的安装路径
SAP_PATHS = (
    r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\SAP Front End\SAP Logon Pad.lnk",
    r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe",
    r"C:\Program Files\SAP\FrontEnd\SAPgui\saplogon.exe",
    r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\SAP Front End.sap",
)

# 报表配置 (使用元组而非字典列表，内存更高效)
REPORTS = (
    ("Z025_01", "SAP_FA LIST_01_Z025"),
    ("Z025_05", "SAP_FA LIST_05_Z025"),
    ("ZTXT_05", "SAP_FA LIST_05_ZTXT"),
    ("Z233_05", "SAP_FA LIST_05_Z233"),
    ("Z503_05", "SAP_FA LIST_05_Z503"),
)

# 月份映射 (财年顺序)
MONTH_FOLDER_MAP = {
    10: "01 October", 11: "02 November", 12: "03 December",
    1: "04 January", 2: "05 February", 3: "06 March",
    4: "07 April", 5: "08 May", 6: "09 June",
    7: "10 July", 8: "11 August", 9: "12 September",
}

# 延时配置 (集中管理，便于调优)
DELAY_SHORT = 0.3      # 短操作
DELAY_MEDIUM = 0.5     # 中等操作
DELAY_LONG = 1.5       # 需要等待的操作
DELAY_EXECUTE = 2.5    # 报表执行等待


# ============== HELPER FUNCTIONS ==============
def close_excel() -> bool:
    """强制关闭所有 Excel 进程"""
    try:
        result = subprocess.run(
            ["taskkill", "/F", "/IM", "EXCEL.EXE"],
            capture_output=True, timeout=5, text=True
        )
        stdout_lower = result.stdout.lower()
        if "成功" in result.stdout or "success" in stdout_lower:
            print("✓ 已关闭 Excel")
            return True
        if "找不到" in result.stdout or "not found" in stdout_lower:
            print("✓ Excel 未运行")
            return True
        return False
    except Exception as e:
        print(f"⚠ 关闭 Excel 时出错: {e}")
        return False


def get_date_info() -> tuple[str, str, str]:
    """
    获取日期相关信息 (预计算，避免循环中重复调用)
    返回: (end_date_formatted, month_folder, year_month_str)
    """
    now = datetime.now()
    year, month = now.year, now.month
    last_day = monthrange(year, month)[1]
    
    end_date = f"{month:02d}/{last_day:02d}/{year:04d}"
    month_folder = MONTH_FOLDER_MAP.get(month, f"{month:02d} Month")
    year_month_str = now.strftime("%Y%m")
    
    return end_date, month_folder, year_month_str


def find_and_start_sap() -> Optional[object]:
    """查找并启动 SAP GUI，返回 SapGuiAuto 对象"""
    # 检查是否已运行
    try:
        sap_auto = win32com.client.GetObject("SAPGUI")
        print("✓ SAP GUI 已在运行")
        return sap_auto
    except Exception:
        pass
    
    print("SAP GUI 未运行，正在查找启动文件...")
    
    # 尝试启动
    for sap_path in SAP_PATHS:
        if os.path.exists(sap_path):
            print(f"找到: {sap_path}")
            print("正在启动 SAP GUI...")
            subprocess.Popen([sap_path], shell=True)
            time.sleep(5)
            
            try:
                return win32com.client.GetObject("SAPGUI")
            except Exception:
                print("✗ SAP GUI 启动失败")
                return None
    
    print("✗ 未找到SAP GUI启动文件")
    print("\n已尝试以下路径:")
    for path in SAP_PATHS:
        print(f"  - {path}")
    print("\n请手动启动SAP GUI并登录")
    return None


def wait_for_login(connection, timeout: int = 60) -> Optional[object]:
    """等待用户登录，返回 session 对象"""
    print("请在弹出的登录窗口中输入用户名和密码...")
    
    for i in range(timeout):
        time.sleep(1)
        if connection.Children.Count > 0:
            session = connection.Children(0)
            print(f"\n✓✓✓ 登录成功！")
            print(f"用户: {session.Info.User}")
            print(f"系统: {session.Info.SystemName}")
            print(f"客户端: {session.Info.Client}")
            return session
        if (i + 1) % 10 == 0:
            print(f"等待登录中... ({i + 1}秒)")
    
    print("⚠ 等待超时，请手动登录后重新运行")
    return None


def try_set_variant(session, variant: str) -> bool:
    """尝试设置变式，返回是否成功"""
    # 方法1: 直接输入
    try:
        session.findById("wnd[0]/usr/ctxtPA_VARI").text = variant
        time.sleep(DELAY_SHORT)
        print(f"✓ 已输入变式 {variant}")
        return True
    except Exception:
        pass
    
    # 方法2: F4帮助
    try:
        session.findById("wnd[0]/tbar[1]/btn[17]").press()
        time.sleep(DELAY_MEDIUM)
        session.findById("wnd[1]/usr/txtV-LOW").text = variant
        session.findById("wnd[1]/usr/txtV-LOW").caretPosition = len(variant)
        time.sleep(DELAY_SHORT)
        session.findById("wnd[1]/tbar[0]/btn[8]").press()
        time.sleep(DELAY_LONG)
        print(f"✓ 已通过F4选择变式 {variant}")
        return True
    except Exception:
        print("⚠ 无法输入变式，尝试直接执行...")
        return False


def try_set_date(session, end_date: str) -> bool:
    """尝试设置日期，返回是否成功"""
    # 方法1: 直接输入 BERDATUM
    try:
        session.findById("wnd[0]/usr/ctxtBERDATUM").text = end_date
        time.sleep(DELAY_SHORT)
        print(f"✓ 已设置日期为: {end_date}")
        return True
    except Exception:
        pass
    
    # 方法2: PA_STIDA
    try:
        session.findById("wnd[0]/usr/ctxtPA_STIDA").text = end_date
        time.sleep(DELAY_SHORT)
        print(f"✓ 已设置日期为: {end_date} (PA_STIDA)")
        return True
    except Exception:
        pass
    
    # 方法3: 日历控件
    try:
        field = session.findById("wnd[0]/usr/ctxtBERDATUM")
        field.setFocus()
        field.caretPosition = 10
        session.findById("wnd[0]").sendVKey(4)
        time.sleep(DELAY_MEDIUM)
        
        calendar = session.findById("wnd[1]/usr/cntlCONTAINER/shellcont/shell")
        calendar.focusDate = end_date
        calendar.selectionInterval = f"{end_date},{end_date}"
        time.sleep(DELAY_SHORT)
        
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
        time.sleep(DELAY_SHORT)
        print(f"✓ 已通过日历设置日期: {end_date}")
        return True
    except Exception:
        pass
    
    print("ℹ 未找到日期字段，将使用变式默认日期")
    return False


def close_popup_windows(session):
    """关闭所有弹出窗口"""
    try:
        while session.Children.Count > 1:
            session.findById("wnd[1]").close()
            time.sleep(DELAY_SHORT)
    except Exception:
        pass


def download_report(session, variant: str, filename: str, save_path: str) -> bool:
    """下载单个报表"""
    # 设置变式
    try_set_variant(session, variant)
    
    # 设置日期 (使用预计算的值)
    try_set_date(session, END_DATE)
    
    time.sleep(DELAY_SHORT)
    
    # 执行报表
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    time.sleep(DELAY_EXECUTE)
    print("✓ 报表正在执行...")
    
    # 导出菜单
    session.findById("wnd[0]/mbar/menu[0]/menu[1]/menu[1]").select()
    time.sleep(DELAY_MEDIUM)
    print("✓ 已打开导出菜单")
    
    # 设置文件名
    filename_field = session.findById(
        "wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_GUI_CUL_EXPORT_AS:0512/txtGS_EXPORT-FILE_NAME"
    )
    filename_field.text = filename
    filename_field.caretPosition = len(filename)
    time.sleep(DELAY_SHORT)
    print(f"✓ 已设置文件名: {filename}")
    
    # 继续到路径选择
    session.findById("wnd[1]/tbar[0]/btn[20]").press()
    time.sleep(DELAY_MEDIUM)
    
    # 打开路径选择
    path_field = session.findById("wnd[1]/usr/ctxtDY_PATH")
    path_field.setFocus()
    path_field.caretPosition = 0
    session.findById("wnd[1]").sendVKey(4)
    time.sleep(DELAY_SHORT)
    
    # 设置保存路径
    path_input = session.findById("wnd[2]/usr/ctxtDY_PATH")
    path_input.text = save_path
    path_input.setFocus()
    path_input.caretPosition = len(save_path)
    time.sleep(DELAY_SHORT)
    
    # 确认路径
    session.findById("wnd[2]/tbar[0]/btn[0]").press()
    time.sleep(DELAY_SHORT)
    print(f"✓ 保存路径: {save_path}")
    
    # 设置后台保存模式
    for checkbox_id in ("wnd[1]/usr/chxDY_NO_REPLACE", "wnd[1]/usr/ctxtDY_NO_REPLACE"):
        try:
            session.findById(checkbox_id).selected = True
            print("✓ 已设置为后台保存模式")
            break
        except Exception:
            continue
    
    time.sleep(DELAY_SHORT)
    
    # 生成文件
    session.findById("wnd[1]/tbar[0]/btn[11]").press()
    time.sleep(DELAY_EXECUTE)
    
    print(f"✓ 文件已保存: {save_path}\\{filename}.xlsx")
    
    # 清理
    close_popup_windows(session)
    
    # 返回选择屏幕
    try:
        session.findById("wnd[0]/tbar[0]/btn[3]").press()
        time.sleep(DELAY_SHORT)
        print("✓ 已返回选择屏幕")
    except Exception:
        print("⚠ 无法返回选择屏幕")
    
    return True


# ============== MAIN EXECUTION ==============
# 预计算日期信息 (避免循环中重复计算)
END_DATE, MONTH_FOLDER, YEAR_MONTH_STR = get_date_info()
SAVE_PATH = f"{BASE_SAVE_PATH}\\{MONTH_FOLDER}\\FA Ledger\\Report\\SAP report {YEAR_MONTH_STR}"

try:
    # 获取 SAP GUI
    SapGuiAuto = find_and_start_sap()
    if not SapGuiAuto:
        sys.exit(1)
    
    application = SapGuiAuto.GetScriptingEngine
    
    # 获取或创建连接
    if application.Children.Count > 0:
        connection = application.Children(0)
        if connection.Children.Count > 0:
            session = connection.Children(0)
            print(f"✓ 已登录 - 用户: {session.Info.User}, 系统: {session.Info.SystemName}")
        else:
            print("⚠ 有连接但未登录，请手动登录")
            sys.exit(1)
    else:
        print(f"\n正在连接到 {SAP_SERVER}...")
        try:
            connection = application.OpenConnection(SAP_SERVER, True)
            print(f"✓ 已发起连接到 {SAP_SERVER}")
            session = wait_for_login(connection)
            if not session:
                sys.exit(1)
        except Exception as e:
            print(f"✗ 连接失败: {e}")
            print("\n请手动操作：")
            print(f"1. 在SAP Logon窗口中双击 '{SAP_SERVER}'")
            print("2. 输入用户名和密码登录")
            sys.exit(1)
    
    print("\n" + "=" * 60)
    print("开始执行报表下载自动化")
    print(f"日期: {END_DATE} | 保存目录: {MONTH_FOLDER}")
    print("=" * 60)
    
    # 最大化并进入事务
    session.findById("wnd[0]").maximize()
    time.sleep(DELAY_SHORT)
    
    session.findById("wnd[0]/tbar[0]/okcd").text = SAP_TRANSACTION
    session.findById("wnd[0]").sendVKey(0)
    time.sleep(DELAY_EXECUTE)
    print(f"✓ 已进入事务代码 {SAP_TRANSACTION}")
    
    # 下载所有报表
    success_count = 0
    for idx, (variant, filename) in enumerate(REPORTS, 1):
        print(f"\n{'=' * 60}")
        print(f"下载报表 {idx}/{len(REPORTS)}: {variant}")
        print("=" * 60)
        
        try:
            if download_report(session, variant, filename, SAVE_PATH):
                success_count += 1
        except Exception as e:
            print(f"✗ 下载失败: {e}")
        
        # 关闭 Excel
        close_excel()
        
        if idx < len(REPORTS):
            time.sleep(DELAY_MEDIUM)
    
    print(f"\n{'=' * 60}")
    print(f"✓✓✓ 完成！成功下载 {success_count}/{len(REPORTS)} 个报表")
    print("=" * 60)

except Exception as e:
    print(f"\n✗ 发生错误: {e}")
    print(f"错误类型: {type(e).__name__}")
    print("\n详细错误信息:")
    traceback.print_exc()
 